<a href="https://colab.research.google.com/github/mefrem/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/module1-statistics-probability-and-inference/LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [33]:
### YOUR CODE STARTS HERE
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2019-09-16 21:59:23--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data.1’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.04s   

2019-09-16 21:59:23 (463 KB/s) - ‘house-votes-84.data.1’ saved [18171/18171]



In [0]:
import pandas as pd
import numpy as np
import scipy.stats

df = pd.read_csv('house-votes-84.data',
               header=None,
               names=['party','handicapped-infants','water-project',
                        'budget','physician-fee-freeze', 'el-salvador-aid',
                        'religious-groups','anti-satellite-ban',
                        'aid-to-contras','mx-missile','immigration',
                        'synfuels', 'education', 'right-to-sue','crime','duty-free',
                        'south-africa'])

In [35]:
# Cleaning data
df = df.replace({'?':np.NaN,'n':0,'y':1})
df.head(1)

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0


In [0]:
# Making my 'samples'
dem = df[df['party'] == 'democrat']
rep = df[df['party'] == 'republican']

In [45]:
# Running one two-sample t-test on 'immigration'
scipy.stats.ttest_ind(dem['immigration'],rep['immigration'],nan_policy='omit',equal_var=False)
# Ttest_indResult(statistic=-1.7374703659666726, pvalue=0.08318508907246192)
# Here it seems that we fail to reject the null hypothesis that there is a
# relationship between the two samples

Ttest_indResult(statistic=-1.7374703659666726, pvalue=0.08318508907246192)

In [72]:
df.head(100)

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
9,democrat,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


### Assignment Answers

I immediately went the path of writing a loop to return two-sample t-test results for each "issue" in the dataset.

After the test is run for each issue, if the p-value is significant (less than 0.05) then I would print statements that associated certain t-statistics with overwhelming yes-votes for an issue by party. A positive t-statistic means the the left argument, in my for-loop the Democratic party, voted more 1s (yes-votes) than the Republicans. Vice versa, if the t-statistic is negative, then the right argument, or Republicans, voted highly (yes-votes).

If the p-values were not significant, I printed a custom statement indicating so.


In [70]:
# Making a for loop that return two-tailed t-test results for each issue
issues =['handicapped-infants','water-project',
                        'budget','physician-fee-freeze', 'el-salvador-aid',
                        'religious-groups','anti-satellite-ban',
                        'aid-to-contras','mx-missile','immigration',
                        'synfuels', 'education', 'right-to-sue','crime','duty-free',
                        'south-africa']

for issue in issues:
  print('')
  print(issue)
  dems = dem[issue]
  reps = rep[issue]
  print(scipy.stats.ttest_ind(dems,reps,nan_policy='omit',equal_var=False))
  if scipy.stats.ttest_ind(dems,reps,nan_policy='omit',equal_var=False)[1] < 0.05:
    if scipy.stats.ttest_ind(dems,reps,nan_policy='omit',equal_var=False)[0] > 0:
      print('We reject the null and find that Democrats voted yes for this issue')
      print('p-value is: ', scipy.stats.ttest_ind(dems,reps,nan_policy='omit',equal_var=False)[1])
    else:
      print('We reject the null and find that Republicans voted yes for this issue')
      print('p-value is: ', scipy.stats.ttest_ind(dems,reps,nan_policy='omit',equal_var=False)[1])
  else:
    print('We fail to reject the null that there was a highly partisan difference in the vote for', issue)


handicapped-infants
Ttest_indResult(statistic=9.662336936353906, pvalue=5.429018745046274e-20)
We reject the null and find that Democrats voted yes for this issue
p-value is:  5.429018745046274e-20

water-project
Ttest_indResult(statistic=-0.08893998898558053, pvalue=0.9291867875225105)
We fail to reject the null that there was a highly partisan difference in the vote for water-project

budget
Ttest_indResult(statistic=22.79324627068853, pvalue=1.9467037396514675e-69)
We reject the null and find that Democrats voted yes for this issue
p-value is:  1.9467037396514675e-69

physician-fee-freeze
Ttest_indResult(statistic=-56.70480591963001, pvalue=8.82076091455592e-193)
We reject the null and find that Republicans voted yes for this issue
p-value is:  8.82076091455592e-193

el-salvador-aid
Ttest_indResult(statistic=-23.907287619273422, pvalue=2.8983726123873854e-79)
We reject the null and find that Republicans voted yes for this issue
p-value is:  2.8983726123873854e-79

religious-groups


### Some Issues

```
aid-to-contras
Ttest_indResult(statistic=18.254856278479288, pvalue=2.2881206640097267e-52)
We reject the null and find that Democrats voted yes for this issue
p-value is:  2.2881206640097267e-52
```

In [80]:
# For Aid to Contras, Democrats voted overwhelmingly in favor of the bill. 
crosstab1 = pd.crosstab(index=df['party'],columns=df['aid-to-contras'])
crosstab1

aid-to-contras,0.0,1.0
party,,
democrat,45,218
republican,133,24


```
crime
Ttest_indResult(statistic=-19.92765596959604, pvalue=6.862816276906505e-58)
We reject the null and find that Republicans voted yes for this issue
p-value is:  6.862816276906505e-58
```

In [81]:
# For Crime, Republicans voted overwhelmingly in favor 
crosstab2 = pd.crosstab(index=df['party'],columns=df['crime'])
crosstab2

crime,0.0,1.0
party,,
democrat,167,90
republican,3,158


```
water-project
Ttest_indResult(statistic=-0.08893998898558053, pvalue=0.9291867875225105)
We fail to reject the null that there was a highly partisan difference in the vote for water-project
```

In [82]:
# For Water Project, there was no clear partisan position or relationship between party and vote
crosstab3 = pd.crosstab(index=df['party'],columns=df['water-project'])
crosstab3

water-project,0.0,1.0
party,,
democrat,119,120
republican,73,75
